# Creating a substitution matrix

We are going to create substitution matrices from sequence alignments. 

## BABA sequences

We will start with the toy example that I showed in the presentation.

We will store the substitution matrix as a dictionary of dictionaries so that we can ask for the score of aligning S with L as *subst_mat['S']['L']*.

In [1]:
from Bio import SeqIO

records = list(SeqIO.parse('baba.fasta', "fasta"))
aa_alphabet = "ABC"  # this is our limitted amino acid alphabet

freqs = {}  # will contain the total count/frequency of each AA
trans = {}  # will contain the total count/frequency of each transition

In [2]:
# we initialize counts
freqs = {aa: 0 for aa in aa_alphabet}
trans = {aa1: {aa2: 0 for aa2 in aa_alphabet} for aa1 in aa_alphabet}

In [3]:
# count all the AA occurences - exclude gaps!
# Iterate over each sequence in records
for record in records:
    # Count occurrences of each amino acid in the sequence
    for aa in record.seq:
        if aa != '-':  # Exclude gaps
            freqs[aa] += 1

In [4]:
# turn the amino acid counts into relative frequencies
total_aa_count = sum(freqs.values())
for aa in freqs:
    freqs[aa] /= total_aa_count


In [5]:
# count transitions
# Iterate over each sequence in records
for record in records:
    # Iterate over each pair of adjacent amino acids in the sequence
    for i in range(len(record.seq) - 1):
        aa1 = record.seq[i]
        aa2 = record.seq[i + 1]
        
        # Exclude transitions involving gaps
        if aa1 != '-' and aa2 != '-':
            # Increment the count of the transition from aa1 to aa2
            trans[aa1][aa2] += 1


In [6]:
# turn transition counts into relative frequencies
# Calculate total number of observed transitions
total_transitions = sum(sum(trans[aa1].values()) for aa1 in aa_alphabet)

# Iterate over each amino acid pair in the transition matrix
for aa1 in aa_alphabet:
    for aa2 in aa_alphabet:
        # Calculate relative frequency for each transition
        if total_transitions != 0:
            trans[aa1][aa2] /= total_transitions

In [7]:
# compute expected frequencies
# Initialize a dictionary to store expected frequencies
expected_freqs = {}

# Iterate over each amino acid pair in the transition matrix
for aa1 in aa_alphabet:
    expected_freqs[aa1] = {}
    for aa2 in aa_alphabet:
        # Calculate expected frequency as the product of individual amino acid frequencies
        expected_freqs[aa1][aa2] = freqs[aa1] * freqs[aa2]


In [11]:
import math

# Initialize a dictionary to store log-odds scores
subst_mat = {}

pseudocount = 1

# Iterate over each amino acid pair in the transition matrix
for aa1 in aa_alphabet:
    subst_mat[aa1] = {}
    for aa2 in aa_alphabet:
        # Compute the log-odds score using the observed and expected frequencies
        observed_freq = trans[aa1][aa2] if (aa1 in trans) and (aa2 in trans[aa1]) else 0
        expected_freq = freqs[aa1] * freqs[aa2]
        
        # Add pseudocount to observed frequency to avoid division by zero
        observed_freq += pseudocount
        
        # Calculate log-odds score
        log_odds_score = math.log10(observed_freq / expected_freq) * 10
        
        # Assign log-odds score to subst_mat
        subst_mat[aa1][aa2] = int(round(log_odds_score, 0))

print(subst_mat)


{'A': {'A': 6, 'B': 11, 'C': 9}, 'B': {'A': 11, 'B': 16, 'C': 14}, 'C': {'A': 8, 'B': 14, 'C': 12}}


We will use pandas to visualize the matrix more easily

In [12]:
import pandas as pd

# Assuming subst_mat is already defined with log-odds scores
df = pd.DataFrame(subst_mat)

# If you want to specify row and column labels explicitly
# df = pd.DataFrame(subst_mat, index=list("ABC"), columns=list("ABC"))

print(df)


    A   B   C
A   6  11   8
B  11  16  14
C   9  14  12


The result above should be the same as with Biopython. 

## GPCR sequences

Here you will create a substitution matrix with real sequences. Use *gpcrs.fasta*. The problem with this alignment is that some columns contain a lot of gaps, which is something undesired to compute a substitution matrix. We will discard columns that contain gaps in >10% of the sequences. 

### Exercise: ungap_fasta

In [15]:
from Bio import SeqIO

def ungap_fasta(input_file, output_file, min_perc=10):
    # Read sequences from input file
    records = []
    with open(input_file, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            try:
                records.append(record)
            except Exception as e:
                print(f"Skipping sequence {record.id} due to error: {e}")
                print(f"Sequence content: {record.seq}")

    # Count gaps at each column
    gap_counts = [0] * len(records[0].seq)
    for record in records:
        for i, char in enumerate(record.seq):
            if char == '-':
                gap_counts[i] += 1

    # Compute percentage of gaps for each column
    total_sequences = len(records)
    gap_percentage = [(count / total_sequences) * 100 for count in gap_counts]

    # Remove columns with more than min_perc% gaps
    out_records = []
    for record in records:
        out_seq = ''.join(char for char, perc in zip(record.seq, gap_percentage) if perc <= min_perc)
        out_records.append(record.__class__(out_seq, id=record.id, name=record.name, description=record.description))

    # Write sequences to output file without removed columns
    with open(output_file, "w") as fname_out:
        SeqIO.write(out_records, fname_out, "fasta")

# Call ungap_fasta function with input and output file paths and minimum percentage
ungap_fasta('gpcrs.fasta', 'gpcrs_ungapped.fasta', 10)



TypeError: SeqRecord (id=5HT1A_HUMAN/1-401) has an invalid sequence.

Use your functions to ungap the gpcrs alignment and compute a sequence alignment with it. Visualize the new substitution matrix in pandas.

In [ ]:
import pandas as pd
aa_alphabet = "ACDEFGHIKLMNPQRSTVWY"
df = pd.DataFrame(subst_mat, list(aa_alphabet), list(aa_alphabet))
df

Let's visualize it using pandas

Compare the matrix with the matrix that you would have obtained without removing the positions with many gaps.

In [ ]:
subst_mat_manygaps = compute_subst_mat('gpcrs.fasta')
df_manygaps = pd.DataFrame(subst_mat_manygaps, list(aa_alphabet), list(aa_alphabet))
df - df_manygaps